In [ ]:
import paper_dataloaders
import paper_models

from seqdata.core import *
from seqdata.models.core import *
from seqdata.learner import *
from seqdata.dataloaders import *
from seqdata.dualrnn import *
from seqdata.hpopt import *
from fastai.basics import *
from fastai.callback.all import *
from seqdata.quaternions import *
from pathlib import Path
from ray import tune
import pickle

from paper_dataloaders import *
from paper_models import *

import ray
from ray import tune
from fastai.basics import *
import pandas as pd
%matplotlib widget

# Analyse the table

In [ ]:
#ray.init(address='auto') #auto is for attaching to a running local server
ray.init() #auto is for attaching to a running local server


RuntimeError: Maybe you called ray.init twice by accident? This error can be suppressed by passing in 'ignore_reinit_error=True' or by calling 'ray.shutdown()' prior to 'ray.init()'.

In [ ]:
experiment_path = Path.home() / 'ray_results/FranSys_P4C_hpopt'
#experiment_path = Path.home() / 'ray_results/FranSys_P4C_hpopt_long'
experiments = get_files(experiment_path,'.json').filter(lambda x: 'experiment_state' in x.name)
# experiment_path = Path.home() / 'ray_results/FranSys_P4C_hpopt_isolated'
# experiments = experiments + get_files(experiment_path,'.json').filter(lambda x: 'experiment_state' in x.name)
experiment_list = [tune.ExperimentAnalysis(e,default_metric='_rmse',default_mode='min') for e in experiments]
df = pd.concat([e.dataframe() for e in experiment_list],ignore_index=True).sort_values('_rmse')
df = df[df.training_iteration == 100].copy()
trials = list(flatten([exp.trials for exp in experiment_list]))
df['trial'] = df.trial_id.apply(lambda t_id: first(trials, lambda y: y.trial_id == t_id))

2024-03-21 07:56:43,317	WARNING experiment_analysis.py:193 -- Failed to fetch metrics for 880 trial(s):
- learner_optimize_d317e_00841: FileNotFoundError('Could not fetch metrics for learner_optimize_d317e_00841: both result.json and progress.csv were not found at /home/pheenix/ray_results/FranSys_P4C_hpopt/learner_optimize_d317e_00841_841_ablation=1,p_diag_loss=0.5055,sync_type=mse_2024-03-20_23-59-24')
- learner_optimize_d317e_00813: FileNotFoundError('Could not fetch metrics for learner_optimize_d317e_00813: both result.json and progress.csv were not found at /home/pheenix/ray_results/FranSys_P4C_hpopt/learner_optimize_d317e_00813_813_ablation=0,p_diag_loss=0.2200,sync_type=mse_2024-03-20_23-59-24')
- learner_optimize_d317e_00831: FileNotFoundError('Could not fetch metrics for learner_optimize_d317e_00831: both result.json and progress.csv were not found at /home/pheenix/ray_results/FranSys_P4C_hpopt/learner_optimize_d317e_00831_831_ablation=0,p_diag_loss=0.0273,sync_type=mse_2024-0

In [ ]:
df.keys()

Index(['train_loss', 'valid_loss', '_rmse', 'timestamp', 'checkpoint_dir_name',
       'should_checkpoint', 'done', 'training_iteration', 'trial_id', 'date',
       'time_this_iter_s', 'time_total_s', 'pid', 'hostname', 'node_ip',
       'time_since_restore', 'iterations_since_restore', 'config/ablation',
       'config/with_state_sync', 'config/with_diag_loss', 'config/sync_type',
       'config/p_state_sync', 'config/p_diag_loss', 'config/pred_sz',
       'config/schedule_pred', 'config/dl', 'config/model', 'config/size',
       'config/pct_start', 'config/n_epoch', 'config/wd', 'config/tsr_alpha',
       'config/tsr_beta', 'config/hidden_p', 'config/has_hidden_p',
       'config/weight_p', 'config/has_weight_p', 'config/opt_beta',
       'config/lr', 'config/create_lrn', 'config/dls', 'config/fit_method',
       'logdir', 'config/with_tar_loss', 'config/init_pred', 'config/p_left',
       'config/p_right', 'trial'],
      dtype='object')

In [ ]:
config = ['config/dl','config/model', 'config/size']
hyperparameters = ['config/sync_type','config/p_state_sync','config/p_diag_loss','config/with_state_sync','config/with_diag_loss']
important_columns = config + hyperparameters + ['train_loss','valid_loss','_rmse','time_total_s']

In [ ]:
top5_per_group = df.groupby(config).apply(lambda x: x.sort_values('_rmse').head(3)).reset_index(drop=True).sort_values(config + ['_rmse'])
top5_per_group[important_columns]

,config/dl,config/model,config/size,config/sync_type,config/p_state_sync,config/p_diag_loss,config/with_state_sync,config/with_diag_loss,train_loss,valid_loss,_rmse,time_total_s
0,Quadrotor,nargru,large,mae,5.539830e+01,0.015244,True,True,0.124564,0.239002,0.370581,7554.061995
1,Quadrotor,nargru,large,mae,1.003709e+04,0.022514,False,True,0.119738,0.238448,0.371428,7369.470818
2,Quadrotor,nargru,large,mae,2.471569e+03,0.008093,False,True,0.120417,0.239419,0.371992,7369.200830
3,Quadrotor,nargru,medium,mse,1.786030e+03,0.053752,False,True,0.110399,0.245758,0.380414,2207.082316
4,Quadrotor,nargru,medium,mse,3.627231e+04,0.001961,False,True,0.110286,0.246381,0.381988,2206.591127
5,Quadrotor,nargru,medium,mse,1.411147e+05,0.014411,False,True,0.110160,0.245465,0.382051,2217.307357
6,Quadrotor,nargru,small,cos,4.159324e+01,0.012785,False,True,0.135856,0.248191,0.380474,1093.649560
7,Quadrotor,nargru,small,mse,2.904675e+04,6.319124,True,False,0.139956,0.250498,0.383052,1263.831056
8,Quadrotor,nargru,small,mse,7.433847e+03,0.001796,True,True,0.137102,0.250699,0.383193,1177.232920
9,Robot,nargru,large,mae,2.412318e+03,7.365002,True,False,0.999971,8.771779,13.966102,6508.909129


In [ ]:
df_ss = df[df['config/with_state_sync']].loc[df[df['config/with_state_sync']].groupby(config)._rmse.idxmin()].sort_values(config,ascending=False).reset_index()
df_dl = df[df['config/with_diag_loss']].loc[df[df['config/with_diag_loss']].groupby(config)._rmse.idxmin()].sort_values(config,ascending=False).reset_index()
df_ss[important_columns]

,config/dl,config/model,config/size,config/sync_type,config/p_state_sync,config/p_diag_loss,config/with_state_sync,config/with_diag_loss,train_loss,valid_loss,_rmse,time_total_s
0,Ship,nargru,small,cos_pow,6.153847,0.031615,True,True,0.039216,0.023875,0.045574,510.851036
1,Ship,nargru,medium,cos_pow,34.500132,0.081620,True,True,0.040935,0.023331,0.044285,653.248340
2,Ship,nargru,large,mse,179266.670706,0.001507,True,False,0.038728,0.023801,0.045374,4661.493569
3,Robot,nargru,small,cos,0.040264,0.812652,True,False,2.411841,9.121016,14.428983,732.220577
4,Robot,nargru,medium,cos_pow,0.412502,1.709704,True,False,0.966508,8.883284,13.880257,1965.302924
5,Robot,nargru,large,mae,2412.318384,7.365002,True,False,0.999971,8.771779,13.966102,6508.909129
6,Quadrotor,nargru,small,mse,29046.749128,6.319124,True,False,0.139956,0.250498,0.383052,1263.831056
7,Quadrotor,nargru,medium,mae,14.880011,0.021379,True,True,0.112585,0.245406,0.382601,2373.034945
8,Quadrotor,nargru,large,mae,55.398305,0.015244,True,True,0.124564,0.239002,0.370581,7554.061995


In [ ]:
best_runs = df.loc[df.groupby(config)._rmse.idxmin()].sort_values(config,ascending=False).reset_index().copy()
best_runs[important_columns]

,config/dl,config/model,config/size,config/sync_type,config/p_state_sync,config/p_diag_loss,config/with_state_sync,config/with_diag_loss,train_loss,valid_loss,_rmse,time_total_s
0,Ship,nargru,small,cos_pow,6.153847,0.031615,True,True,0.039216,0.023875,0.045574,510.851036
1,Ship,nargru,medium,cos_pow,34.500132,0.081620,True,True,0.040935,0.023331,0.044285,653.248340
2,Ship,nargru,large,mse,179266.670706,0.001507,True,False,0.038728,0.023801,0.045374,4661.493569
3,Robot,nargru,small,cos,0.040264,0.812652,True,False,2.411841,9.121016,14.428983,732.220577
4,Robot,nargru,medium,cos_pow,0.412502,1.709704,True,False,0.966508,8.883284,13.880257,1965.302924
5,Robot,nargru,large,mae,2412.318384,7.365002,True,False,0.999971,8.771779,13.966102,6508.909129
6,Quadrotor,nargru,small,cos,41.593236,0.012785,False,True,0.135856,0.248191,0.380474,1093.649560
7,Quadrotor,nargru,medium,mse,1786.030426,0.053752,False,True,0.110399,0.245758,0.380414,2207.082316
8,Quadrotor,nargru,large,mae,55.398305,0.015244,True,True,0.124564,0.239002,0.370581,7554.061995


In [ ]:
best_runs['config/p_state_sync'] = df_ss['config/p_state_sync']
best_runs['config/sync_type'] = df_ss['config/sync_type']
best_runs['config/p_diag_loss'] = df_dl['config/p_diag_loss']

In [ ]:
best_runs[important_columns]

,config/dl,config/model,config/size,config/sync_type,config/p_state_sync,config/p_diag_loss,config/with_state_sync,config/with_diag_loss,train_loss,valid_loss,_rmse,time_total_s
0,Ship,nargru,small,cos_pow,6.153847,0.031615,True,True,0.039216,0.023875,0.045574,510.851036
1,Ship,nargru,medium,cos_pow,34.500132,0.081620,True,True,0.040935,0.023331,0.044285,653.248340
2,Ship,nargru,large,mse,179266.670706,0.070765,True,False,0.038728,0.023801,0.045374,4661.493569
3,Robot,nargru,small,cos,0.040264,0.038614,True,False,2.411841,9.121016,14.428983,732.220577
4,Robot,nargru,medium,cos_pow,0.412502,0.108142,True,False,0.966508,8.883284,13.880257,1965.302924
5,Robot,nargru,large,mae,2412.318384,0.030997,True,False,0.999971,8.771779,13.966102,6508.909129
6,Quadrotor,nargru,small,mse,29046.749128,0.012785,False,True,0.135856,0.248191,0.380474,1093.649560
7,Quadrotor,nargru,medium,mae,14.880011,0.053752,False,True,0.110399,0.245758,0.380414,2207.082316
8,Quadrotor,nargru,large,mae,55.398305,0.015244,True,True,0.124564,0.239002,0.370581,7554.061995


In [ ]:
keys_config = [x for x in best_runs.keys() if 'config' in x]
# hpopt_configs = {tuple(row[config]): dict(row[hyperparameters+config]) for (idx,row) in best_runs.iterrows()}
hpopt_configs = {tuple(row[config]): {k[7:]:dict(row[hyperparameters])[k] for k in dict(row[hyperparameters])} for (idx,row) in best_runs.iterrows()}
hpopt_configs

{('Ship', 'nargru', 'small'): {'sync_type': 'cos_pow',
  'p_state_sync': 6.153846756954221,
  'p_diag_loss': 0.03161509785732224,
  'with_state_sync': True,
  'with_diag_loss': True},
 ('Ship', 'nargru', 'medium'): {'sync_type': 'cos_pow',
  'p_state_sync': 34.50013244819111,
  'p_diag_loss': 0.08161991365158368,
  'with_state_sync': True,
  'with_diag_loss': True},
 ('Ship', 'nargru', 'large'): {'sync_type': 'mse',
  'p_state_sync': 179266.6707063948,
  'p_diag_loss': 0.07076493307625548,
  'with_state_sync': True,
  'with_diag_loss': False},
 ('Robot', 'nargru', 'small'): {'sync_type': 'cos',
  'p_state_sync': 0.04026387041667635,
  'p_diag_loss': 0.03861371289229526,
  'with_state_sync': True,
  'with_diag_loss': False},
 ('Robot', 'nargru', 'medium'): {'sync_type': 'cos_pow',
  'p_state_sync': 0.4125019527820889,
  'p_diag_loss': 0.10814233762957079,
  'with_state_sync': True,
  'with_diag_loss': False},
 ('Robot', 'nargru', 'large'): {'sync_type': 'mae',
  'p_state_sync': 2412.318

In [ ]:
import pickle
pickle.dump(hpopt_configs, open('configs_4C.p', 'wb') )